In [4]:

import pandas as pd
import matplotlib.pyplot as plt

# Paths - adjust if your notebook is in a different location
DATA_DIR = "dataset/CodeWorkout"

In [5]:

main_table = pd.read_csv(f"{DATA_DIR}/MainTable.csv")
print(f"Main table: {len(main_table):,} rows")

codestate_table = pd.read_csv(f"{DATA_DIR}/LinkTables/CodeStates.csv")
print(f"CodeState table: {len(codestate_table):,} rows")

subject_table = pd.read_csv(f"{DATA_DIR}/LinkTables/Subject.csv")
print(f"Subject table: {len(subject_table):,} rows")

Main table: 201,570 rows
CodeState table: 69,627 rows
Subject table: 381 rows


In [6]:

df = main_table.merge(codestate_table, on="CodeStateID")
df = df.merge(subject_table, on="SubjectID")
print(f"Joined dataset: {len(df):,} rows")
print(f"Columns: {df.columns.tolist()}")

Joined dataset: 191,584 rows
Columns: ['Order', 'SubjectID', 'ToolInstances', 'ServerTimestamp', 'ServerTimezone', 'CourseID', 'CourseSectionID', 'TermID', 'AssignmentID', 'ProblemID', 'Attempt', 'CodeStateID', 'IsEventOrderingConsistent', 'EventType', 'Score', 'Compile.Result', 'CompileMessageType', 'CompileMessageData', 'EventID', 'ParentEventID', 'SourceLocation', 'Code', 'X-Grade']


In [7]:
df.head()

,Order,SubjectID,ToolInstances,ServerTimestamp,ServerTimezone,CourseID,CourseSectionID,TermID,AssignmentID,ProblemID,...,EventType,Score,Compile.Result,CompileMessageType,CompileMessageData,EventID,ParentEventID,SourceLocation,Code,X-Grade
0,0,14502,Java 8; CodeWorkout,2019-03-20T19:34:25,0,CS 1114,410.0,spring-2019,492.0,32,...,Run.Program,1.0,NaN,NaN,NaN,32-84779,NaN,NaN,"public String plusOut(String str, String word)...",0.88
1,1,14502,Java 8; CodeWorkout,2019-03-20T19:34:25,0,CS 1114,410.0,spring-2019,492.0,32,...,Compile,NaN,Success,NaN,NaN,32-84780,32-84779,NaN,"public String plusOut(String str, String word)...",0.88
2,4,13499,Java 8; CodeWorkout,2019-03-20T21:13:09,0,CS 1114,410.0,spring-2019,492.0,32,...,Run.Program,0.0,NaN,NaN,NaN,32-84783,NaN,NaN,"public String plusOut(String str, String word)...",0.92
3,5,13499,Java 8; CodeWorkout,2019-03-20T21:13:09,0,CS 1114,410.0,spring-2019,492.0,32,...,Compile,NaN,Error,NaN,NaN,32-84784,32-84783,NaN,"public String plusOut(String str, String word)...",0.92
4,6,13499,Java 8; CodeWorkout,2019-03-20T21:13:09,0,CS 1114,410.0,spring-2019,492.0,32,...,Compile.Error,NaN,NaN,SyntaxError,line 3: error: ';' expected,32-84785,32-84784,Text:3,"public String plusOut(String str, String word)...",0.92


In [8]:
print("Submissions per Problem:")
submissions_per_problem = df.groupby('ProblemID').size().sort_values(ascending=False)
print(submissions_per_problem.describe())
print(f"\nUnique problems: {df['ProblemID'].nunique()}")

Submissions per Problem:
count       50.00000
mean      3831.68000
std       2203.66066
min       1277.00000
25%       2467.50000
50%       3201.50000
75%       4095.00000
max      13388.00000
dtype: float64

Unique problems: 50


In [9]:
# Classify each submission (by CodeStateID)
def classify_submission(group):
    event_types = group['EventType'].values
    
    if 'Run.Program' not in event_types:
        return 'Not Run'
    elif group[group['EventType'] == 'Run.Program']['Score'].max() == 1.0:
        return 'Success'
    elif 'Compile.Error' in event_types:
        return 'Compile Error'
    else:
        return 'Ran But Failed Tests'

# Get one row per submission (Run.Program event or first event if no run)
def get_submission_summary(group):
    run_events = group[group['EventType'] == 'Run.Program']
    if not run_events.empty:
        return run_events.iloc[0]
    else:
        return group.iloc[0]

# Summarize each submission
submissions = df.groupby('CodeStateID').apply(get_submission_summary).reset_index(drop=True)

# Add submission status
status = df.groupby('CodeStateID').apply(classify_submission)
submissions['Status'] = submissions['CodeStateID'].map(status)

# Add compile error message if exists
compile_errors = df[df['EventType'] == 'Compile.Error'][['CodeStateID', 'CompileMessageData']]
compile_errors = compile_errors.groupby('CodeStateID')['CompileMessageData'].first()
submissions['ErrorMessage'] = submissions['CodeStateID'].map(compile_errors)

# Get BEST attempt per student-problem
def get_best_attempt(group):
    priority = {'Success': 1, 'Ran But Failed Tests': 2, 'Compile Error': 3, 'Not Run': 4}
    group = group.copy()
    group['Priority'] = group['Status'].map(priority)
    group = group.sort_values(['Priority', 'Score', 'Attempt'], ascending=[True, False, False])
    return group.iloc[0]

best_attempts = submissions.groupby(['SubjectID', 'ProblemID']).apply(get_best_attempt).reset_index(drop=True)

# Select useful columns
best_attempts = best_attempts[[
    'SubjectID', 
    'AssignmentID', 
    'ProblemID', 
    'Attempt', 
    'Score', 
    'Status',
    'ErrorMessage',
    'CodeStateID',
    'Code',
    'X-Grade',
    'ServerTimestamp'
]]

# Summary
print(f"Total student-problem pairs: {len(best_attempts):,}")
print(f"Unique students: {best_attempts['SubjectID'].nunique()}")
print(f"Unique problems: {best_attempts['ProblemID'].nunique()}")
print(f"\nStatus breakdown:")
print(best_attempts['Status'].value_counts())

best_attempts.head(10)

/tmp/ipykernel_1016/2070633239.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  submissions = df.groupby('CodeStateID').apply(get_submission_summary).reset_index(drop=True)
/tmp/ipykernel_1016/2070633239.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  status = df.groupby('CodeStateID').apply(classify_submission)


Total student-problem pairs: 15,375
Unique students: 372
Unique problems: 50

Status breakdown:
Status
Success                 14286
Ran But Failed Tests     1004
Compile Error              85
Name: count, dtype: int64


/tmp/ipykernel_1016/2070633239.py:42: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_attempts = submissions.groupby(['SubjectID', 'ProblemID']).apply(get_best_attempt).reset_index(drop=True)


,SubjectID,AssignmentID,ProblemID,Attempt,Score,Status,ErrorMessage,CodeStateID,Code,X-Grade,ServerTimestamp
0,106,439.0,1,6,1.0,Success,NaN,1741398,"public int sortaSum(int a, int b)\r\n{\r\n ...",0.81,2019-02-24T23:52:21
1,106,439.0,3,3,1.0,Success,NaN,1741564,"public boolean in1To10(int n, boolean outsideM...",0.81,2019-02-24T23:59:04
2,106,439.0,5,2,1.0,Success,NaN,1741226,"public boolean answerCell(boolean isMorning, b...",0.81,2019-02-24T23:44:51
3,106,439.0,12,2,1.0,Success,NaN,1741678,"public boolean squirrelPlay(int temp, boolean ...",0.81,2019-02-25T00:03:06
4,106,439.0,13,1,1.0,Success,NaN,1740821,"public int caughtSpeeding(int speed, boolean i...",0.81,2019-02-24T23:23:05
5,106,487.0,17,8,1.0,Success,NaN,1789095,"public int redTicket(int a, int b, int c)\r\n{...",0.81,2019-03-09T01:16:11
6,106,487.0,20,3,1.0,Success,NaN,1788998,"public int loneSum(int a, int b, int c)\r\n{\r...",0.81,2019-03-09T01:02:51
7,106,487.0,21,1,1.0,Success,NaN,1788982,"public int luckySum(int a, int b, int c)\r\n{\...",0.81,2019-03-09T01:00:11
8,106,487.0,22,26,1.0,Success,NaN,1789119,"public int noTeenSum(int a, int b, int c)\r\n{...",0.81,2019-03-09T01:20:21
9,106,487.0,24,6,1.0,Success,NaN,1788650,"public int blackjack(int a, int b)\r\n{\r\n ...",0.81,2019-03-08T23:56:20


In [12]:
# Assignment order (from dataset documentation)
assignment_order = [439.0, 487.0, 492.0, 494.0, 502.0]

# Early = First 3 assignments (30 problems) - for analysis
# Late = Last 2 assignments (20 problems) - for prediction/validation
early_assignments = [439.0, 487.0, 492.0]
late_assignments = [494.0, 502.0]



In [40]:
# This is just a FILTER, not a new calculation
early_attempts = best_attempts[best_attempts['AssignmentID'].isin(early_assignments)].copy()
late_attempts = best_attempts[best_attempts['AssignmentID'].isin(late_assignments)].copy()

# Verify
print(f"best_attempts: {len(best_attempts):,} rows")
print(f"early_attempts: {len(early_attempts):,} rows")
print(f"late_attempts: {len(late_attempts):,} rows")
print(f"Sum: {len(early_attempts) + len(late_attempts):,} (should equal best_attempts)")

best_attempts: 15,375 rows
early_attempts: 9,568 rows
late_attempts: 5,807 rows
Sum: 15,375 (should equal best_attempts)


In [41]:

early_attempts = best_attempts[best_attempts['AssignmentID'].isin(early_assignments)].copy()
late_attempts = best_attempts[best_attempts['AssignmentID'].isin(late_assignments)].copy()

# Get problem lists
early_problems = early_attempts['ProblemID'].unique().tolist()
late_problems = late_attempts['ProblemID'].unique().tolist()

print("=" * 60)
print("EARLY vs LATE SPLIT")
print("=" * 60)
print(f"\nEarly Assignments: {early_assignments}")
print(f"Late Assignments: {late_assignments}")
print(f"\nEarly Problems ({len(early_problems)}): {sorted(early_problems)}")
print(f"Late Problems ({len(late_problems)}): {sorted(late_problems)}")
print(f"\nEarly Attempts: {len(early_attempts):,} student-problem pairs")
print(f"Late Attempts: {len(late_attempts):,} student-problem pairs")
print(f"Total: {len(early_attempts) + len(late_attempts):,} (should equal {len(best_attempts):,})")


EARLY vs LATE SPLIT

Early Assignments: [439.0, 487.0, 492.0]
Late Assignments: [494.0, 502.0]

Early Problems (30): [1, 3, 5, 12, 13, 17, 20, 21, 22, 24, 25, 28, 31, 32, 33, 34, 36, 37, 38, 39, 40, 100, 101, 102, 128, 232, 233, 234, 235, 236]
Late Problems (20): [41, 43, 44, 45, 46, 48, 49, 51, 56, 57, 64, 67, 70, 71, 104, 106, 107, 108, 112, 118]

Early Attempts: 9,568 student-problem pairs
Late Attempts: 5,807 student-problem pairs
Total: 15,375 (should equal 15,375)
